In [1]:
import numpy as np
import pickle as pk
import pandas as pd
import timeit as tm
import csv
import sys

In [2]:
# Open training data to pandas
train_dat_pandas = pd.read_csv('../data/clean_data/sum/train_vectors.csv', index_col=0, encoding='utf-8')

# Open training labels to pandas
train_lbl_pandas = pd.read_csv('../data/clean_data/sum/train_labels.csv', index_col=0, encoding='utf-8')

# Save headers
headers = [list(train_dat_pandas)]

# Convert pandas to numpy matrix
train_dat = train_dat_pandas.as_matrix()
print 'training data dimensions:', train_dat.shape

# Convert pandas to numpy matrix
train_lbl = train_lbl_pandas.as_matrix()
print 'training label dimensions:', train_lbl.shape




training data dimensions: (225159, 63)
training label dimensions: (225159, 1)


In [3]:
# Open test data
test_dat_pandas = pd.read_csv('../data/clean_data/sum/test_vectors.csv', index_col=0, encoding='utf-8')

# Open test labels
test_lbl_pandas = pd.read_csv('../data/clean_data/sum/test_labels.csv', index_col=0, encoding='utf-8')

# Convert pandas to numpy matrix
test_dat = test_dat_pandas.as_matrix()
print 'testing data dimensions:', test_dat.shape

# Convert pandas to numpy matrix
test_lbl = test_lbl_pandas.as_matrix()
print 'testing label dimensions:', test_lbl.shape

testing data dimensions: (25537, 63)
testing label dimensions: (25537, 1)


In [4]:
full_dat_pandas = pd.concat([train_dat_pandas, test_dat_pandas])
full_dat = full_dat_pandas.as_matrix()

full_lbl_pandas = pd.concat([train_lbl_pandas, test_lbl_pandas])
full_lbl = full_lbl_pandas.as_matrix()

In [5]:
# Feature vector scalling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(train_dat)
train_dat = scaler.transform(train_dat)
print train_dat.shape
test_dat = scaler.transform(test_dat)
print test_dat.shape

scaler.fit_transform(full_dat)

(225159, 63)
(25537, 63)


array([[-1.43904909, -1.63615815, -2.57352052, ..., -0.68743182,
        -0.99726662,  2.58747393],
       [-1.43904909, -1.63615815, -2.56793824, ..., -0.68743182,
        -0.99726662,  2.58747393],
       [-1.43904909, -1.63615815, -2.55945282, ..., -0.68743182,
        -0.99726662,  2.58747393],
       ..., 
       [ 1.57735425,  1.59781705,  1.73165128, ..., -0.87581872,
        -1.03421317,  1.54054648],
       [ 1.57735425,  1.59781705,  1.74930845, ..., -0.87581872,
        -1.03421317,  1.54054648],
       [ 1.57735425,  1.59781705,  2.26035083, ..., -0.87581872,
        -1.03421317,  1.54054648]])

In [ ]:
from sklearn.linear_model import LinearRegression

# Fit Linear Regression
lin_reg = LinearRegression(n_jobs=-1, normalize=True)
lin_reg.fit(train_dat, train_lbl)

In [ ]:
# Generate predictions
predictions = lin_reg.predict(test_dat)
print predictions.shape

In [ ]:
# Compute RMSE

import math

errors = []

# compute squared errors
for i in xrange(predictions.shape[0]):
    p = predictions[i]
    t = test_lbl[i]
    
    # compute distance
    squared_distance = 0.0
    for j in xrange(predictions.shape[1]):
        squared_distance += (p[j] - t[j])**2
    
    errors.append(squared_distance)

rmse = math.sqrt(sum(errors)/len(errors))
print 'Root mean squared error:', rmse

In [ ]:
print lin_reg.score(test_dat, test_lbl)

In [6]:
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor(n_jobs=-1, n_estimators=10, verbose=2)
reg.fit(train_dat, train_lbl)


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   21.8s finished


building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
           verbose=2, warm_start=False)

In [7]:
print reg.score(test_dat, test_lbl)

0.349106445068


[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished


In [8]:
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

parameters = {'max_depth':[None, 2, 4, 8, 16, 32, 64], 'max_features':['sqrt', 'log2', None]}
reg_internal = RandomForestRegressor()

reg = GridSearchCV(reg_internal, parameters, n_jobs=-1, cv=5, pre_dispatch='n_jobs', refit=True, verbose=1)
reg.fit(train_dat, train_lbl)

Fitting 5 folds for each of 21 candidates, totalling 105 fits


/Library/Python/2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/Library/Python/2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/Library/Python/2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/Library/Python/2.7/site-

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_features': ['sqrt', 'log2', None], 'max_depth': [None, 2, 4, 8, 16, 32, 64]},
       pre_dispatch='n_jobs', refit=True, scoring=None, verbose=1)

In [9]:
print 'Score on test data:', reg.score(test_dat, test_lbl)

print 'best params:', reg.best_params_

Score on test data: 0.434897397868
best params: {'max_features': 'sqrt', 'max_depth': 16}


In [10]:
predictions = reg.predict(test_dat)
print predictions

[ 1.1710643   1.20605193  1.19818727 ...,  1.16429578  2.91666667  3.24      ]


In [11]:
from sklearn.ensemble import RandomForestRegressor
champion = RandomForestRegressor(n_jobs=-1, 
                                 max_depth=reg.best_params_['max_depth'], 
                                 max_features=reg.best_params_['max_features'])
champion.fit(full_dat, full_lbl)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=16,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [12]:
# Save model
from sklearn.externals import joblib

joblib.dump(champion, '../models/RandomForestRegressor.p')

['../models/RandomForestRegressor.p',
 '../models/RandomForestRegressor.p_01.npy',
 '../models/RandomForestRegressor.p_02.npy',
 '../models/RandomForestRegressor.p_03.npy',
 '../models/RandomForestRegressor.p_04.npy',
 '../models/RandomForestRegressor.p_05.npy',
 '../models/RandomForestRegressor.p_06.npy',
 '../models/RandomForestRegressor.p_07.npy',
 '../models/RandomForestRegressor.p_08.npy',
 '../models/RandomForestRegressor.p_09.npy',
 '../models/RandomForestRegressor.p_10.npy',
 '../models/RandomForestRegressor.p_11.npy',
 '../models/RandomForestRegressor.p_12.npy',
 '../models/RandomForestRegressor.p_13.npy',
 '../models/RandomForestRegressor.p_14.npy',
 '../models/RandomForestRegressor.p_15.npy',
 '../models/RandomForestRegressor.p_16.npy',
 '../models/RandomForestRegressor.p_17.npy',
 '../models/RandomForestRegressor.p_18.npy',
 '../models/RandomForestRegressor.p_19.npy',
 '../models/RandomForestRegressor.p_20.npy',
 '../models/RandomForestRegressor.p_21.npy',
 '../models/Rando